In [6]:
import pandas as pd
electoral_result = pd.read_csv('/workspaces/Electoral-Economic-Analysis/electoral_analysis/1976-2020-president.csv')

electoral_result = electoral_result.drop(columns=['state_po', 'state_fips', 'state_cen', 'state_ic', 'writein', 'notes', 'version'])

electoral_result = electoral_result[electoral_result['year'] >= 2008]

electoral_result.to_csv('/workspaces/Electoral-Economic-Analysis/electoral_analysis/2008-2020-president.csv')
print(electoral_result.head())


      year    state        office         candidate party_detailed  \
2728  2008  ALABAMA  US PRESIDENT      MCCAIN, JOHN     REPUBLICAN   
2729  2008  ALABAMA  US PRESIDENT  OBAMA, BARACK H.       DEMOCRAT   
2730  2008  ALABAMA  US PRESIDENT               NaN    INDEPENDENT   
2731  2008  ALABAMA  US PRESIDENT               NaN            NaN   
2732  2008   ALASKA  US PRESIDENT      MCCAIN, JOHN     REPUBLICAN   

      candidatevotes  totalvotes party_simplified  
2728         1266546     2099819       REPUBLICAN  
2729          813479     2099819         DEMOCRAT  
2730           16089     2099819            OTHER  
2731            3705     2099819            OTHER  
2732          193841      326197       REPUBLICAN  
